### [1] Module, Variables

In [1]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC  # noqa

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [77]:
# Account Variables

info = open('secure/id_pw.txt').readlines()

account_1 = info[0].split(' ')[1].rstrip('\n')
password_1 = info[1].split(' ')[1].rstrip('\n')

account_2 = info[2].split(' ')[1].rstrip('\n')
password_2 = info[3].split(' ')[1].rstrip('\n')

account_in = account_1
password_in = password_2 


### [2] Access, Sign-in

In [50]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [51]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
try:
    btn_sign_in = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
    btn_sign_in.click()
except:
    pass

# 계정 이메일 입력
try:
    input_account = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
    input_account.send_keys(account_in)
except:
    pass
# [다음] 버튼 클릭해 패스워드 입력
try:
    btn_next_password = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
    btn_next_password.click()
except:
    pass

# 계정 비밀번호 입력
try:
    input_password = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
    input_password.send_keys(password_in)
    del password_in # 계정 비밀번호 변수 삭제
except:
    pass

# [다음] 버튼 입력해 로그인 클릭
try:
    btn_next_sign_in = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
    btn_next_sign_in.click()
except:
    pass



In [ ]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지
try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


In [41]:
driver.refresh()

### [3] Daily Record Collection

#### 1) Variables & Functions

In [52]:
# Variables (1) - Record data Tag Attributes


# 기본 영상 정보
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>
# WatchDate = container[data-date] / c-wiz 태그의 [data-date] 값

# 시청유무 판별 정보
nm_class_thumbnail = "OUPWA"  # 기록 우측에 있는 영상 정보 (썸네일) / tag = <div>
nm_class_percentage = "HmLFgd"  # 썸네일 하단 빨간 Percentage Bar / tag = <div>
# RunningTime 활용

# 시청기록 - video id
# replace(pattern, '')로 id 값 추출
id_str_pattern = "https://www.youtube.com/watch?v="


In [53]:
# Variables (2) Google API - Youtube Data


# API 실행에 필요한 모듈
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

# API 정보, API Key => API build
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
DEVELOPER_KEY = open('secure/api_key.txt').readline()

youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=DEVELOPER_KEY)

In [54]:
# Function (1) - Datetime


def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d')

def get_datadate(input_date=datetime.today()):
    return datetime.strftime(input_date, '%Y%m%d')

print(get_datadate()) # 오늘 날짜를 data-date 형식으로
get_date(get_datadate())

20230219


datetime.datetime(2023, 2, 19, 0, 0)

In [55]:
# Function (2) - Category Id & Title

""" Google API - Youtube DATA API 활용

    세부 query
    - videos().list(): id 값(watch?v="여기 표시되는 값")으로 동영상 정보를 호출
        - categoryId 호출을 위해 활용
    
    - videoCategories().list(): categoryId 값(1~44)으로 카테고리 정보를 호출
        - categoryTitle 호출을 위해 활용
"""

""" 개선 필요
        categoryTitle이 영어로 호출됨
        query 옵션에서 한국어를 호출하거나, 번역해야 한다.
"""


def get_catId(input_id):
    video = youtube.videos().list(
        part='snippet',
        id=input_id
    ).execute()

    return video['items'][0]['snippet']['categoryId']


def get_catTitle(num):
    catId = youtube.videoCategories().list(
        id=num,
        part='snippet'
    ).execute()

    try:
        result = catId['items'][0]['snippet']['title']
    except:
        result = ''

    return result


In [56]:
# Function (3) - Watched_YN classifier

""" 구현 가능 Option (1)
    - 구현법
        1. 현재 레코드의 watch_time과 이전 레코드의 watch_time의 차이값 구한다
        2. 영상 길이와 비슷하면 watched, 아닐 시 not_watched

    - 문제점
        1. 마지막으로 시청한 영상은 차이를 구할 수 없다.
        2. 길이 5분 이상 광고를 방치한 경우 시간 계산에 오류가 생긴다
"""

""" 구현 가능 Option (2) -> 현재 Option (2) 채택
    - 구현법
        1. 시청 기록 레코드의 썸네일의 진행도 막대를 활용한다.
        2. 시청 완료하지 않은 영상에 표시되며, 태그 내부의 시청 비율 값을 추출한다.

    - 문제점
        1. 라이브 스트리밍의 경우 잠깐 보고 꺼도 진행도 막대가 출력되지 않는다.
        2. 대충 넘기기만 해도 진행도 막대가 표시된다. 
"""


def classify_watched(content):
    running_time = content.find('div', class_=nm_class_RunningTime).text
    thumbnail = content.find('div', class_=nm_class_thumbnail)
    percentage = thumbnail.find('div', class_=nm_class_percentage)

    try:
        result = int(percentage['style'].split(":")[1][:-1]) / 100

    except:
        if len(running_time) > 5: # 영상 길이가 1시간 이상(0:00:00)인 경우
            result = 0 # 미시청으로 간주
        else:
            result = 1 # 시청으로 간주

    return result


#### 2) DataFrame

In [57]:
import pandas as pd
columns_rec = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category_id', 'category_title', 'watched_YN', 'account']
df_record = pd.DataFrame(columns=columns_rec)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched_YN,account


In [58]:
columns_ser = ['searchword', 'search_date', 'search_time', 'account']
df_search = pd.DataFrame(columns=columns_ser)
df_search.head()


,searchword,search_date,search_time,account


#### 3) Web Data Collecting

In [75]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')


호출된 레코드 수: 100


In [76]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):
    
    if content['data-date'] == get_datadate(datetime.today()):

        if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
            if content.find('div', class_=nm_class_RunningTime): # 영상 길이가 있는 경우 - 실제 시청기록
                print(f"{idx}: Watch Record")

                title = content.find('a', class_=nm_class_title).text
                channel = content.find('div', class_=nm_class_channel).text 
                running_time = content.find(
                    'div', class_=nm_class_RunningTime).text
                
                watch_date = get_date(content['data-date'])
                watch_time = ' '.join(
                    (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
                
                video_id = content.find('a')['href'].replace(id_str_pattern, '')
                category_id = get_catId(video_id)
                category_title = get_catTitle(category_id)

                watched = classify_watched(content) 
                

                df_record.loc[idx_rec] = [title, channel, running_time, 
                                        watch_date, watch_time, category_id, category_title, watched, account_1]
                idx_rec += 1

            else:
                print(f"{idx}: Community Post \t Watch time: {watch_time}")

        else:
            if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
                print(f"{idx}: Search Record")

                title = content.find('a', class_=nm_class_title).text
                search_date = get_date(content['data-date'])
                search_time = ' '.join(
                    (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

                df_search.loc[idx_ser] = [title, search_date, search_time, account_1]

                idx_ser += 1

            else:
                print(f"{idx}: Ad")

                num_ad += 1

    else:
        break
    

0: Watch Record
1: Watch Record
2: Watch Record


In [78]:
df_record.to_csv(f"data/{get_datadate()}_watch_record_{account_in}.csv", encoding='utf-8')
df_record.head()

,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched_YN,account
0,[#무한도전] ★설날 특집 150분 순삭★ 레전드의 향연 《명수는 12살》 보면서 ...,오분순삭,2:35:18,2023-02-19,오전 11:50,24,Entertainment,0.1,kjaehw0207
1,한 명만 재미없는 여느 때의 T1│1R LSB vs T1│ 2023 LCK SPRI...,Wolf,1:03:13,2023-02-19,오전 10:44,20,Gaming,0.0,kjaehw0207
2,패드립,팍플레이Pacplay,10:54,2023-02-19,오전 10:31,20,Gaming,1.0,kjaehw0207


In [36]:
df_search.to_csv(f"data/{get_datadate()}_search_record_{account_in}.csv", encoding='utf-8')
df_search.head()

,searchword,search_date,search_time,account


In [79]:
driver.close()